This part of the code inizializes functions for the Ising 1d model: 

In [3]:
%matplotlib inline
from __future__ import division
import numpy as np
from numpy.random import rand
import random
import matplotlib.pyplot as plt
import csv

def initialstate(N):   
    ''' generates a random spin configuration for initial condition'''
    state = 2*np.random.randint(2, size=(N))-1
    return state


def mcmove(config, beta, J, h):
    '''Monte Carlo move using Metropolis algorithm '''
    for i in range(N):
                a = np.random.randint(0, N)
                s =  config[a]
                nb = config[(a+1)%N] + config[(a-1)%N] 
                cost = 2*s* (J*nb + h)   
                if cost < 0:
                    s *= -1
                elif rand() < np.exp(-cost*beta):
                    s *= -1
                config[a] = s
    return config


def calcEnergy(config,J,h):
    '''Energy of a given configuration'''
    energy = 0
    for i in range(len(config)):
            S = config[i]
            nb = config[(i+1)%N] + config[(i-1)%N] 
            energy += -J*nb*S -h*S
    return energy/2. 


def calcMag(config):
    '''Magnetization of a given configuration'''
    mag = np.sum(config)
    return mag


nt      = 1         #  number of temperature points
N       = 200         #  size of the lattice, N 
eqSteps = 1024       #  number of MC sweeps for equilibration
mcSteps = 5000       #  number of MC sweeps for calculation

J       = 1.0
h       = 1.0
T       = np.linspace(3.28, 3.29, nt); #1.53
E,M,C,X = np.zeros(nt), np.zeros(nt), np.zeros(nt), np.zeros(nt)
n1, n2  = 1.0/(mcSteps*N), 1.0/(mcSteps*mcSteps*N*N) 
# divide by number of samples, and by system size to get intensive values

 Run the following cell only if you want to produce cnfgs and store them into a folder. If you already have the cnfg, skip this part.  

In [ ]:
#----------------------------------------------------------------------
#  MAIN PART OF THE CODE
#----------------------------------------------------------------------
for tt in range(nt):
    E1 = M1 = E2 = M2 = 0
    config = initialstate(N)
    iT=1.0/T[tt]; iT2=iT*iT;
    
    for i in range(eqSteps):             # equilibrate
        mcmove(config, iT,J,h)           # Monte Carlo moves

    for i in range(mcSteps):
        mcmove(config, iT,J,h)           
        Ene = calcEnergy(config,J,h)     # calculate the energy
        Mag = calcMag(config)            # calculate the magnetisation

        E1 = E1 + Ene
        M1 = M1 + Mag
        #M2 = M2 + Mag*Mag 
        #E2 = E2 + Ene*Ene
        
        parameters = [Ene/N, Mag/N, T[tt]]
        np.savetxt("./cnfg_1d/cnfg_1d_T" + str(T[tt]) + "_" + str(i) + ".csv", config.astype(int),fmt='%i',delimiter=",") 
        with open("./cnfg_1d/cnfg_1d_T" + str(T[tt]) + "_" + str(i) + ".csv", 'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(parameters)
        csvFile.close()
        
    E[tt] = n1*E1
    M[tt] = n1*M1
    #C[tt] = (n1*E2 - n2*E1*E1)*iT2
    #X[tt] = (n1*M2 - n2*M1*M1)*iT
    
    
f = plt.figure(figsize=(18, 10)); # plot the calculated values    

sp =  f.add_subplot(2, 2, 1 );
plt.scatter(T, E, s=50, marker='o', color='IndianRed')
plt.xlabel("Temperature (T)", fontsize=20);
plt.ylabel("Energy ", fontsize=20);         plt.axis('tight');

sp =  f.add_subplot(2, 2, 2 );
plt.scatter(T, abs(M), s=50, marker='o', color='RoyalBlue')
plt.xlabel("Temperature (T)", fontsize=20); 
plt.ylabel("Magnetization ", fontsize=20);   plt.axis('tight');

#sp =  f.add_subplot(2, 2, 3 );
#plt.scatter(T, C, s=50, marker='o', color='IndianRed')
#plt.xlabel("Temperature (T)", fontsize=20);  
#plt.ylabel("Specific Heat ", fontsize=20);   plt.axis('tight');   

#sp =  f.add_subplot(2, 2, 4 );
#plt.scatter(T, X, s=50, marker='o', color='RoyalBlue')
#plt.xlabel("Temperature (T)", fontsize=20); 
#plt.ylabel("Susceptibility", fontsize=20);   plt.axis('tight');    

Now read from saved cnfg

In [4]:
cnfg_in=np.zeros((nt,mcSteps,N))
params_in=np.zeros((nt,mcSteps,3))        
for tt in range(nt):
    for i in range(mcSteps):
        cnfg_in[tt,i,:] = np.genfromtxt("./cnfg_1d/cnfg_1d_T" + str(T[tt]) + "_" + str(i) + ".csv", dtype=float, delimiter='\t', skip_footer=1) 
        params_in[tt,i,:] = np.genfromtxt("./cnfg_1d/cnfg_1d_T" + str(T[tt]) + "_" + str(i) + ".csv", dtype=float, delimiter=',', skip_header=N) 

Now Keras/TensorFlow. The idea is to use a RNN with LSTM cells to "learn" how a certain cnfg of 1d ising is ordered.
Notice that this is conceptually wrong, since the spin chain is NOT a time series... however let us see what happens if we try to generate new sequences which correspond to a label $[T_i,M_i,E_i]$ where $T$ is the temperature, $M$ the magnetization and $E$ the energy of a given cnfg $i$.

In [5]:
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector, Bidirectional, Conv1D, BatchNormalization
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.backend import one_hot
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from time import time

X = to_categorical(0.5*(cnfg_in[0,:,:]+1))   #if not one-hot encoder does not work with [1,-1]
shift=1        #Wonder what happens if I put 0... I should discover Identity transformation...
perc=0.0                                     #I do validation split directly into keras model...

Y = np.roll(X,shift, axis=0)  #forward shift: X(t)=Y(t-shift)  One might want to do a backward shift as well ...
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=perc, random_state=1)
Y_train = np.swapaxes(Y_train, 0, 1)  # this is weird ... But I follow Sacred Mr. Ng 
Y_test = np.swapaxes(Y_test, 0, 1)   # this is weird ... But I follow Sacred Mr. Ng

Using TensorFlow backend.


Define Keras model: $T_x$ is the total length N of our cnfg, $n_a$ is the number of activations, $n_{\rm values}$=2 since is a $\mathbf{Z}_2$ symmetry model. 

In [6]:
n_states=X.shape[2]  # possible spin projections. Ready for Potts model.
n_a=50             # number of activation units at each time t 

reshapor = Reshape((1, n_states))                
LSTM_cell = LSTM(n_a, return_state = True, activation='tanh')   #testing linear activation --> require clipping 
#Conv_cell = Conv1D(filters=1,strides=1, padding="same", kernel_size=3) #linear act
densor = Dense(n_states, activation='softmax')   

In [11]:
def Ising1d_model(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values ? 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X  = Input(shape=(Tx, n_values))
    #X1 = Conv1D(filters=2,strides=1,padding='same',kernel_size=3, activation='tanh')(X);
    #X1 = BatchNormalization()(X1)
    #X1 = Activation('tanh')(X1)
    #X1 = Dropout(0.8)(X1)
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0

    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        i=t-1
        f=t+2
        if t-1<0:
            i=0
            f=3
        if t+1>Tx-1:
            i=Tx-4
            f=Tx-1
        
        
        xtmp = Lambda(lambda x: X[:,i:f,:])(X)
        X1   = Conv1D(filters=2,strides=1,padding='valid',kernel_size=3, activation='tanh')(xtmp);

        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        X1 = reshapor(X1)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(X1, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X, a0, c0], outputs=outputs)
    
    return model

In [12]:
model = Ising1d_model(N, n_a, n_states)
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

m =  int(mcSteps*(1.0-perc))
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200, 2)       0                                            
__________________________________________________________________________________________________
lambda_203 (Lambda)             (None, 3, 2)         0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_203 (Conv1D)             (None, 1, 2)         14          lambda_203[0][0]                 
__________________________________________________________________________________________________
lambda_204 (Lambda)             (None, 3, 2)         0           input_3[0][0]                    
__________________________________________________________________________________________________
reshape_1 

In [ ]:
history = model.fit([X_train, a0, c0], list(Y_train), epochs=1, verbose=1, validation_split=0.1)#, callbacks=[tbCallBack])

Train on 4500 samples, validate on 500 samples
Epoch 1/1


In [ ]:
#plots 
f = plt.figure(figsize=(18, 10)); # plot the calculated values    
sp =  f.add_subplot(2, 2, 1 );
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper right')

sp =  f.add_subplot(2, 2, 2 );
plt.plot(history.history['dense_1_loss'])
plt.plot(history.history['val_dense_1_loss'])
plt.title('model loss')
plt.ylabel('dense loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper right')

sp =  f.add_subplot(2, 2, 3 );
plt.plot(history.history['dense_1_acc'])
plt.plot(history.history['val_dense_1_acc'])
plt.title('model acc')
plt.ylabel('dense acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='lower right')

In [ ]:
def Ising_inference_model(LSTM_cell, densor, n_values, n_a, Ty):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(Ty, n_values))
    X1 = Conv1D(filters=2,strides=1,padding='same',kernel_size=3, activation='tanh')(x0);
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = X1
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)
        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        x[:,t,:] = Lambda(one_hot)(out)

    inference_model = Model(inputs=[x0, a0, c0], outputs=outputs)
    
    return inference_model

In [ ]:
inference_model = Ising_inference_model(LSTM_cell, densor, n_values = 2, n_a = n_a , Ty = N)
X_tmp = np.random.randint(2,size=(1, N))
x_initializer = to_categorical(X_tmp) 
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

In [ ]:
def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer, a_initializer, c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = np.argmax(pred, axis=-1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (1, )
    results = to_categorical(indices, num_classes=2)
 
    
    return results, indices, pred

In [ ]:
_ , _, proba = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)

In [ ]:
def sample_cnfg(prob,seed):
    shape = np.shape(prob)
    sampled_cnfg = np.zeros((shape[0],1))
    
    for i in range(shape[0]):
        random.seed(seed+i*shape[0])
        sampled_cnfg[i] = np.random.choice(list(range(shape[2])),p=prob[i][0][:]) 
    return sampled_cnfg

In [ ]:
Nrep = 5000
new_cnfg = np.zeros((Nrep,N)) 
for i in range(Nrep):
    tmp = sample_cnfg(proba,i)
    new_cnfg[i,:] = tmp[:,0]           

Now I compare the energy and magnetization distribution among the MC and RNN data: 

In [ ]:
E_MC  = np.zeros((N))
M_MC  = np.zeros((N))

E_RNN = np.zeros((Nrep))
M_RNN = np.zeros((Nrep))


for i in range(N): 
    E_MC[i]  = calcEnergy(cnfg_in[0,i,:],J,h)
    M_MC[i]  = calcMag(cnfg_in[0,i,:])

for i in range(Nrep): 
    new_cnfg[i,:][new_cnfg[i,:]<1] = -1
    E_RNN[i] = calcEnergy(new_cnfg[i,:],J,h)
    M_RNN[i] = calcMag(new_cnfg[i,:])

    
f = plt.figure(figsize=(18, 10)); # plot the calculated values    
sp =  f.add_subplot(2, 2, 1 );
plt.hist(E_MC, bins=10, normed=1, alpha=0.5, color='red', label='MC')
plt.hist(E_RNN[2:], bins=10, normed=1, alpha=0.5, color='blue', label='RNN')
plt.legend(loc='upper right')
plt.xlabel("Energy", fontsize=20); 


sp =  f.add_subplot(2, 2, 2 );
plt.hist(M_MC, bins=10, normed=1, alpha=0.5, color='red', label='MC')
plt.hist(M_RNN[2:], bins=10,normed=1, alpha=0.5, color='blue', label='RNN')
plt.legend(loc='upper right')
plt.xlabel("Magnetization", fontsize=20);

It seems that Magnetization is correctly reproduced (meaning, the total amount of + and - spins). 
However, the Energy that keeps track of the ordering among the spins is completely screwed up. 